In [2]:
import glob
import os
import pandas as pd
import torch
from ase.constraints import FixSymmetry
from ase.filters import FrechetCellFilter
from ase.io import read, write
from ase.optimize import FIRE
from mattersim.forcefield import MatterSimCalculator
from ase.spacegroup import get_spacegroup

# -------------------------------------------------
# --- CONFIGURATION SECTION ---
# Folder containing input POSCAR files (e.g., in VASP format)
input_folder = './ref2/' 
# Folder to save optimized structures and results
output_folder = './ref2/opt/'
# Maximum force tolerance for relaxation (eV/Angstrom)
fmax_threshold = 0.01
# Tolerance for symmetry detection (Angstrom)
symprec_value = 0.01  
# Optimizer steps limit
max_steps = 2000
# Name of the MatterSim model to load
mattersim_model = "MatterSim-v1.0.0-5M.pth"
# -------------------------------------------------

# --- SETUP ---
# Determine device (CUDA or CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"    # Uncomment to force CPU
print(f"Running MatterSim on {device}")

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

results_list = []

search_path = os.path.join(input_folder, '*_POSCAR*')
poscar_files = glob.glob(search_path)

# Helper function to get Space Group Symbol safely
def get_sg_symbol(atoms_obj):
    """Detects and returns the Space Group Symbol for an ASE Atoms object."""
    try:
        # Use symprec for tolerance in detecting symmetry
        sg = get_spacegroup(atoms_obj, symprec=symprec_value)
        return sg.symbol
    except Exception:
        return "N/A"

if not poscar_files:
    print("No files found matching the pattern.")
else:
    print(f"Found {len(poscar_files)} files. Starting optimization with MatterSim...\n")

# --- MAIN LOOP ---
for filepath in poscar_files:
    base_filename = os.path.basename(filepath)
    print(f"--- Processing: {base_filename} ---")
    
    try:
        # 1. Read Structure
        atoms = read(filepath)
        
        # 2. Get Initial Info (Symmetry)
        initial_sg = get_sg_symbol(atoms)

        # --- CALCULATOR SETUP ---
        # Initialize MatterSim calculator once per structure (or once outside the loop for efficiency)
        # Note: Initializing outside the loop is faster if the model path is the same.
        atoms.calc = MatterSimCalculator(load_path=mattersim_model, device=device)
        
        # 3. Get Initial Energy (Requires a calculator)
            # Need to call get_potential_energy() to calculate initial energy
        initial_energy = atoms.get_potential_energy()   
        # 4. Optimization Setup
        # FixSymmetry can be applied, but it will constrain the cell parameters based on initial symmetry.
        # Remove or adjust this line if you want the structure to break symmetry or the cell to change freely.
        # atoms.set_constraint([FixSymmetry(atoms)]) 
        
        # FrechetCellFilter allows both atomic positions and cell parameters to change
        opt_target = FrechetCellFilter(atoms)
        # Using FIRE as it's a robust optimizer for relaxations
        optimizer = FIRE(opt_target, logfile=None) 
        
        # 5. Run Optimization
        optimizer.run(fmax=fmax_threshold, steps=max_steps) 
        
        # 6. Get Final Info
        final_energy = atoms.get_potential_energy()
        final_sg = get_sg_symbol(atoms)
        delta_energy = final_energy - initial_energy
        
        # 7. Save Optimized Structure
        filename_root, _ = os.path.splitext(base_filename)
        # Save the final relaxed structure from the filter
        new_filename = f"{filename_root}_mattersim_opt"
        save_path = os.path.join(output_folder, new_filename)
        write(save_path, atoms, format='vasp', direct=True) # direct=True for VASP format
        
        # 8. Store Data
        results_list.append({
            'Filename': base_filename,
            'Final_E_Total': final_energy,
            'Atoms': len(atoms),
            'Delta_E': delta_energy,
            'Init_SG': initial_sg,
            'Final_SG': final_sg
        })
        
        print(f"  >> Done. E: {final_energy:.3f} eV | SG: {initial_sg} -> {final_sg}")

    except Exception as e:
        print(f"  >> FATAL Error processing file {base_filename}: {e}")

# --- RANKING & TXT OUTPUT SECTION ---
## 🏆 Generating Ranking Report

if results_list:
    print("\n" + "="*80)
    print("   GENERATING RANKING REPORT   ")
    print("="*80)

    df = pd.DataFrame(results_list)
    
    # Calculate energy per atom, handling potential division by zero
    if 'Atoms' in df.columns and (df['Atoms'] > 0).all():
        df['E_per_Atom'] = df['Final_E_Total'] / df['Atoms']
    else:
        print("Warning: Could not calculate E/Atom. Using Total Energy for ranking.")
        df['E_per_Atom'] = df['Final_E_Total']

    # SORTING: Lowest Energy per Atom (most stable) first
    df_sorted = df.sort_values(by='E_per_Atom', ascending=True).reset_index(drop=True)
    df_sorted.index += 1 
    df_sorted.index.name = 'Rank'

    txt_path = os.path.join(output_folder, 'energy_ranking_mattersim.txt')

    with open(txt_path, 'w', encoding='utf-8') as f:
        # Define the header format
        header = (f"{'Rank':<5} | {'Filename':<25} | {'E/Atom':<10} | "
                  f"{'Delta E':<10} | {'Init SG':<10} | {'Final SG':<10}\n")
        
        f.write("MatterSim Batch Optimization Results\n")
        f.write(f"Model: {mattersim_model}, fmax: {fmax_threshold}\n\n")
        f.write(header)
        f.write("-" * len(header) + "\n")
        
        for rank, row in df_sorted.iterrows():
            # Truncate filename if too long for a clean table
            fname = (row['Filename'][:22] + '..') if len(row['Filename']) > 22 else row['Filename']
            
            line = (f"{rank:<5} | "
                    f"{fname:<25} | "
                    f"{row['E_per_Atom']:<10.4f} | "
                    f"{row['Delta_E']:<10.4f} | "
                    f"{row['Init_SG']:<10} | "
                    f"{row['Final_SG']:<10}\n")
            f.write(line)

    print(f"Ranking saved to: **{txt_path}**")
    print("\nTop 5 Structures:")
    # Print the top 5 results to the console
    print(df_sorted[['Filename', 'E_per_Atom', 'Init_SG', 'Final_SG']].head(5).to_string())

else:
    print("No structures were successfully optimized.")

Running MatterSim on cpu
Found 19 files. Starting optimization with MatterSim...

--- Processing: 7_POSCAR ---


/var/folders/0f/_n97csts5c5c3ldj1k40vc100000gn/T/ipykernel_44981/2325296173.py:47: FutureWarning: `get_spacegroup` has been deprecated due to its misleading output. The returned `Spacegroup` object has symmetry operations for a standard setting regardress of the given `Atoms` object. See https://gitlab.com/ase/ase/-/issues/1534 for details. Please use `ase.spacegroup.symmetrize.check_symmetry` or `spglib` directly to get the symmetry operations for the given `Atoms` object.
  sg = get_spacegroup(atoms_obj, symprec=symprec_value)


  >> Done. E: -80.640 eV | SG: A m m 2 -> C m m m
--- Processing: 47_POSCAR ---
  >> Done. E: -81.174 eV | SG: P m m m -> P 1
--- Processing: 71_POSCAR ---
  >> Done. E: -79.763 eV | SG: R -3 c -> R -3 c
--- Processing: 57_POSCAR ---
  >> Done. E: -80.780 eV | SG: C 2 -> C 2
--- Processing: 10_POSCAR ---
  >> Done. E: -80.243 eV | SG: P n -3 m -> P -1
--- Processing: 9_POSCAR ---
  >> Done. E: -81.949 eV | SG: P 21 21 21 -> P 21 21 21
--- Processing: 88_POSCAR ---
  >> Done. E: -83.033 eV | SG: P 42 c m -> P 1
--- Processing: 94_POSCAR ---
  >> Done. E: -81.809 eV | SG: A e a 2 -> C c c 2
--- Processing: 67_POSCAR ---
  >> Done. E: -80.792 eV | SG: P 42 n m -> P n n m
--- Processing: 51_POSCAR ---
  >> Done. E: -81.234 eV | SG: P m m a -> P 1
--- Processing: 93_POSCAR ---
  >> Done. E: -84.946 eV | SG: P m m m -> P -1
--- Processing: 60_POSCAR ---
  >> Done. E: -78.926 eV | SG: P 4 m m -> P 4 m m
--- Processing: 48_POSCAR ---
  >> Done. E: -80.725 eV | SG: I 21 3 -> I 21 3
--- Processi

In [6]:
import numpy as np
from ase.io import read
from dscribe.descriptors import SOAP
import os

# --- 1. INPUT: Read from POSCAR files ---
file_8 = "standard_/Si16_POSCAR"
file_64 = "standard_/Si_POSCAR"

if not os.path.exists(file_8) or not os.path.exists(file_64):
    print("ERROR: Please make sure Si8_POSCAR and Si64_POSCAR exist.")
    exit()

si_8 = read(file_8)
si_64 = read(file_64)

print(f"Loaded {file_8}: {len(si_8)} atoms")
print(f"Loaded {file_64}: {len(si_64)} atoms")
print("-" * 50)

# --- 2. SETUP: Define SOAP Descriptor ---
# CORRECTED SYNTAX: used underscores (r_cut, n_max, l_max)
soap = SOAP(
    species=["Si"],
    periodic=True,
    r_cut=6.0,
    n_max=4,   
    l_max=4,
    sigma=0.5,     # Added sigma (width of the gaussian) just in case
    average="off"
)

# --- 3. CALCULATION: Generate Vectors ---
vec_8 = soap.create(si_8)
vec_64 = soap.create(si_64)

# --- 4. OUTPUT: Compare Vectors ---
print("SHAPE COMPARISON:")
print(f"Si-8 Vector Shape:  {vec_8.shape}")
print(f"Si-16 Vector Shape: {vec_64.shape}")

# Compare Atom 0 vs Atom 0
v1 = vec_8[0]
v2 = vec_64[0]
diff = np.linalg.norm(v1 - v2)

print("\nVALUE COMPARISON (Atom 0):")
print(f"Si-8  (first 5 numbers): {v1[:20]}")
print(f"Si-64 (first 5 numbers): {v2[:20]}")
print(f"Euclidean Difference:    {diff:.10f}")

if diff < 1e-6:
    print("\n>> RESULT: MATCH.")
else:
    print("\n>> RESULT: MISMATCH.")
# --- 5. SUBTRACT VECTORS (Show the Difference) ---
print("-" * 50)
print("VECTOR SUBTRACTION (v1 - v2):")

# Calculate the difference vector
diff_vector = v1 - v2

# 1. Print the first 10 elements of the difference
# If these are all 0.0, the start of the vector is identical
print("First 10 differences:", diff_vector[:10])

# 2. Check if the WHOLE vector is effectively zero
# We look for the maximum deviation in the entire array
max_diff = np.max(np.abs(diff_vector))

print(f"\nMaximum difference found in any element: {max_diff:.10f}")

if max_diff < 1e-6:
    print(">> The difference vector is effectively ZERO everywhere.")
else:
    print(">> There are non-zero differences.")
    # Optional: Show indices where difference is large
    # This helps find exactly WHICH feature is different
    indices = np.where(np.abs(diff_vector) > 1e-6)[0]
    print(f">> Indices with difference > 1e-6: {indices[:10]} ... (total {len(indices)})")
    print(f">> Value at first mismatch (Index {indices[0]}): {diff_vector[indices[0]]}")

Loaded standard_/Si16_POSCAR: 8 atoms
Loaded standard_/Si_POSCAR: 16 atoms
--------------------------------------------------
SHAPE COMPARISON:
Si-8 Vector Shape:  (8, 50)
Si-16 Vector Shape: (16, 50)

VALUE COMPARISON (Atom 0):
Si-8  (first 5 numbers): [ 4.29067914e-01  1.61920149e+00 -1.26984362e+00  3.32371487e+00
  6.11048594e+00 -4.79209143e+00  1.25429189e+01  3.75815287e+00
 -9.83666681e+00  2.57466945e+01  1.12466913e-32 -3.10417452e-32
  7.79253048e-32 -4.35552346e-32  8.73987885e-32 -2.21592050e-31
  1.10604829e-31  5.67470819e-31 -2.67813829e-31  2.24318760e-31]
Si-64 (first 5 numbers): [ 4.29067914e-01  1.61920149e+00 -1.26984362e+00  3.32371487e+00
  6.11048594e+00 -4.79209143e+00  1.25429189e+01  3.75815287e+00
 -9.83666681e+00  2.57466945e+01  3.63732866e-32 -9.30177201e-32
  1.20579474e-31 -2.09012458e-32  2.37944104e-31 -3.07840012e-31
  5.19259762e-32  4.11716717e-31 -9.31788158e-32  6.47843751e-32]
Euclidean Difference:    0.0000000008

>> RESULT: MATCH.
------------